In [3]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
load_dotenv()

client = OpenAI()

class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})
    
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)
    
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        
        chat_completion = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
        )
        response=chat_completion.choices[0].message.content
        
        self.add_to_memory("assistant", response)
        
        return response

    
agent=ConversationAgent(model="gpt-4")

print(f"---> Agent response:\n\n{agent.run('Hello!')}")

---> Agent response:

Hello! How can I assist you today?
